# Efficient Yelp API Calls CORE
Loveida Lucero

In [4]:
# Standard Imports
import numpy as np
import pandas as pd
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [1]:
with open('/Users/Lovei/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [2]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [3]:
# use our yelp_api variable's search_query method to perform our API call
search_results = yelp_api.search_query(location='Salem, MA',
                                       term='Soup')
print(type(search_results))
search_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [6]:
# set our API call parameters 
LOCATION = 'Salem, MA'
TERM = 'Soup'

In [7]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE2 = "Data/results_in_progress_Salem_Soup.json"
JSON_FILE2

'Data/results_in_progress_Salem_Soup.json'

In [8]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE2)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE2)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE2} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE2,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE2} already exists.")

[i] Data/results_in_progress_Salem_Soup.json not found. Saving empty list to file.


In [9]:
## Load previous results and use len of results for offset
with open(JSON_FILE2,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [10]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [11]:
## How many results total?
total_results = results['total']
total_results

317

In [12]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [13]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

16

In [14]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE2,'w') as f:
     json.dump(previous_results,f)

In [15]:
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/16 [00:00<?, ?it/s]

In [16]:
## delete file and confirm it no longer exits.
os.remove(JSON_FILE2)
os.path.isfile(JSON_FILE2)

False

In [17]:
def create_json_file(JSON_FILE2,  delete_if_exists=False):
    
    ## Check if JSON_FILE2 exists
    file_exists = os.path.isfile(JSON_FILE2)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE2} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE2)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE2,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE2} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE2} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE2)
        
        ## If JSON_FILE2 included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE2,'w') as f:
            json.dump([],f)  

In [20]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE2, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE2,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_Salem_Soup.json not found. Saving empty list to new file.
- 0 previous results found.


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [19]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE2, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE2,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/16 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: 'Data/results_in_progress_Salem_Soup.json'

In [ ]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

In [ ]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

In [ ]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

In [ ]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_Salem_Soup.csv.gz', compression='gzip',index=False)